In [1]:
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from joblib import dump

In [2]:
df_data = pd.read_csv("../../results/processed_dataset/1_processed_input_dataset.csv")
df_data

,Age,Weight,Height,Clycemia,SBP,DBP,Cholesterol,Triglycerides,HDL,LDL,MSPH,BMI
0,21,63.0,158.0,70.0,96.0,64.0,140,75.0,61.0,64.0,0,25.2
1,20,112.0,164.0,77.0,80.0,60.0,126,81.0,36.0,74.0,0,41.6
2,19,77.8,165.0,93.0,102.0,64.0,149,105.0,56.0,73.0,0,28.6
3,29,92.0,165.0,90.0,110.0,70.0,149,144.0,49.0,71.0,0,33.8
4,18,79.0,160.0,66.0,106.0,60.0,167,150.0,22.0,115.0,0,30.9
...,...,...,...,...,...,...,...,...,...,...,...,...
143,30,53.0,156.0,85.1,140.0,88.0,158,55.0,43.5,103.5,0,21.8
144,32,63.0,169.0,74.2,109.0,72.0,238,123.0,84.1,129.3,0,22.1
145,34,66.0,155.0,79.5,104.0,69.0,216,183.0,56.8,122.6,0,27.5
146,37,88.5,171.0,85.2,106.0,53.0,146,97.0,54.0,72.6,0,30.3


In [3]:
df_data["MSPH"].value_counts()

MSPH
0    105
1     43
Name: count, dtype: int64

In [4]:
positive_data = df_data[df_data["MSPH"] == 1]
negative_data = df_data[df_data["MSPH"] == 0]

negative_filter = shuffle(negative_data, n_samples=len(positive_data), random_state=849)
negative_filter.index

Index([ 13,  38, 121,  35,  98, 120, 119, 143, 110, 114,   0,  17, 131, 102,
         3,  40,  34,   6, 128,   4,  28,   9,  30, 112, 123,  44,  20,   8,
       115,  42, 130,  10, 127, 133, 137, 109, 124,  47,  32, 136,  16,  14,
       142],
      dtype='int64')

In [5]:
all_negative_index = negative_data.index

not_use_index = []

for index in all_negative_index:
    if index not in negative_filter.index:
        not_use_index.append(index)

len(not_use_index)

62

In [6]:
Filter_df = df_data[df_data.index.isin(not_use_index)]
Filter_df.to_csv("../../results/processed_dataset/2_data_not_used_to_train.csv", index=False)

In [7]:
df_data_to_train = pd.concat([positive_data, negative_filter], axis=0)
df_train, df_test = train_test_split(df_data_to_train, random_state=849, test_size=.10)
df_test.to_csv("../../results/processed_dataset/3_test_independent_dataset.csv", index=False)
df_train.to_csv("../../results/processed_dataset/3_train_dataset.csv", index=False)

In [8]:
df_values = df_train.drop(columns=["MSPH"])
response = df_train["MSPH"]

In [9]:
scaler = MaxAbsScaler()
scaler.fit(df_values.values)
data_transform = scaler.transform(df_values.values)

In [10]:
df_transfomr = pd.DataFrame(data=data_transform, columns=df_values.columns)
df_transfomr["response"] = response.values

In [11]:
dump(scaler, "../../results/processed_dataset/scaler_instance.joblib")

['../../results/processed_dataset/scaler_instance.joblib']

In [12]:
df_transfomr.to_csv("../../results/processed_dataset/4_data_transform.csv", index=False)

In [13]:
df_transfomr["response"].value_counts()

response
0    39
1    38
Name: count, dtype: int64

In [14]:
df_test["MSPH"].value_counts()

MSPH
1    5
0    4
Name: count, dtype: int64